In [38]:
import os
import copy
import pickle
import pandas as pd
import copy
from statistics import mean

import numpy as np
import scipy
from tqdm import tqdm

from gensim.models import Word2Vec
from gensim import utils
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel

import sys
sys.path.append('../src')
from models import get_matrices_from_term_lists

In [8]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

In [29]:
we_model = Word2Vec.load(we_model_dir+'/model.gensim')

print ('loading done!')

loading done!


In [10]:
#loading E_ctx_vec
with open(we_model_dir+'/E_ctx_vec.pkl', 'rb') as fr:
    E_ctx_vec = pickle.load(fr)
with open(we_model_dir+'/E_wrd_vec.pkl', 'rb') as fr:
    E_wrd_vec = pickle.load(fr)

In [11]:
def get_expSG_vecs(words):
    
    expSG_vecs = {}
    for word in words:
        _idx = we_model.wv.vocab[word].index
        _vec = we_model.wv.vectors[_idx]
        
        # explicit SkipGram
        expSG_vecs[word] = scipy.special.expit(np.dot(we_model.trainables.syn1neg, _vec))
        expSG_vecs[word] /= np.sqrt(E_ctx_vec * E_wrd_vec[_idx])
    
    return expSG_vecs

get_expSG_vecs(['book'])

{'book': array([0.24737692, 0.56118023, 0.68307143, ..., 0.09694416, 0.18521136,
        0.15946515], dtype=float32)}

In [12]:
def get_expSG_1storder_relation(word_from, words_to):
    expSG_vec = get_expSG_vecs([word_from])[word_from]
    
    relations={}
    for word_to in words_to:
        if word_to in we_model.wv.vocab:
            _idx = we_model.wv.vocab[word_to].index
            relations[word_to] = expSG_vec[_idx]
    
    return relations

get_expSG_1storder_relation('book', ['book', 'library'])

{'book': 11.011071, 'library': 3.6249995}

# Calculating *d*

In [35]:
A_terms = ['book','library']
B_terms = ['car','garage']
X_terms = ['female','woman']
Y_terms = ['male','man']

In [46]:
# This function is untested
def get_1storder_association_metric(word, A_terms, B_terms):
    A_relations = get_expSG_1storder_relation(word, A_terms)
    B_relations = get_expSG_1storder_relation(word, B_terms)
    return mean(A_relations.values()) - mean(B_relations.values())
get_1storder_association_metric('book', ['book','library'], ['car','garage'])

6.6768265

In [33]:
get_expSG_1storder_relation('book',['car','garage'])

{'car': 0.3352654, 'garage': 0.9471519}

In [49]:
def produce_1storder_effect_size(X_terms, Y_terms, A_terms, B_terms):
    x_associations = np.array([])
    y_associations = np.array([])
    for (x,y) in zip(X_terms, Y_terms):
        x_association = get_1storder_association_metric(x, A_terms, B_terms)
        y_association = get_1storder_association_metric(y, A_terms, B_terms)
        x_associations = np.append(x_associations, x_association)
        y_associations = np.append(y_associations, y_association)
    all_associations = np.append(x_associations, y_associations)
    return (np.mean(x_associations) - np.mean(y_associations))/np.std(all_associations, ddof=1)
produce_1storder_effect_size(X_terms, Y_terms, A_terms, B_terms)

1.047680254118163

In [48]:
a = np.array([1,2])
b = np.array([3,4])
np.append(a,b)

array([1, 2, 3, 4])